In [1]:
import json
import os
import pandas as pd
import numpy as np
import pychartjs
from datetime import datetime 
import matplotlib.pyplot as plt
import re
import altair as alt
import seaborn as sns
from matplotlib.pyplot import figure

In [24]:
output_dir = "output_data/"
file = "response3.pki"
raw_data = pd.read_pickle(output_dir + file)
raw_data = raw_data[["time", "Log ID", "Machine ID", "full_command", "command", "stderr", "stdout"]]
raw_data

,time,Log ID,Machine ID,full_command,command,stderr,stdout
0,2021-04-19 19:34:00,490cd3ad601e875614199fc11aa5ebe2ad67d289cbe519...,10bcd15c,ls,ls,,SuperShell\nstderr.txt
1,2021-04-19 19:35:00,490cd3ad601e875614199fc11aa5ebe2ad67d289cbe519...,10bcd15c,export $TEST=Hello,export,/home/supershell/.offline.sh: line 358: export...,
2,2021-04-19 19:35:00,490cd3ad601e875614199fc11aa5ebe2ad67d289cbe519...,10bcd15c,export $TEST='Hello',export,/home/supershell/.offline.sh: line 358: export...,
3,2021-04-19 19:36:00,490cd3ad601e875614199fc11aa5ebe2ad67d289cbe519...,10bcd15c,export PATH=$PATH:$HOME/bin,export,,
4,2021-04-19 19:36:00,490cd3ad601e875614199fc11aa5ebe2ad67d289cbe519...,10bcd15c,export PATH=$PATH:test/bin,export,,
...,...,...,...,...,...,...,...
468,2021-04-17 19:57:00,00206cd4bdc39bab7e2908eb900370094d721883f30148...,612c4dc4-ce35-4571-952d-cfedc92809d3-5fbf7c6db6-n,ls,ls,,_15881.env\n_15881.variables\nfastx_out\nfx486...
469,2021-04-17 19:57:00,00206cd4bdc39bab7e2908eb900370094d721883f30148...,612c4dc4-ce35-4571-952d-cfedc92809d3-5fbf7c6db6-n,cd ..,cd,,
470,2021-04-17 19:57:00,00206cd4bdc39bab7e2908eb900370094d721883f30148...,612c4dc4-ce35-4571-952d-cfedc92809d3-5fbf7c6db6-n,ls,ls,,job-48632\njob-48633\njob-48634\njob-48635\njo...
471,2021-04-17 19:57:00,00206cd4bdc39bab7e2908eb900370094d721883f30148...,612c4dc4-ce35-4571-952d-cfedc92809d3-5fbf7c6db6-n,pwd,pwd,,/home/jovyan/DNASubway/project-5974


In [25]:
raw_data['Log ID'] = raw_data['Log ID'].replace({'490cd3ad601e875614199fc11aa5ebe2ad67d289cbe519e3a41d8fbd2f2af05e':'User_1', '33bb76e6130caab90d89f35542de42c1be3a0f364190f6e3645cd8f03613c1ab': 'User_2', '8067d36662ba1f8b6d7e2281fb338c35f016d2f2e72b2e09bb010bd323e89970': 'User_3', '2f3c04c9be3cc02268104e419f0c962bc674e4e7411c8c0f67b34f11f89fc398': 'User_4', 'd255305150a0d892434887745e969de6e48e7d48bc7f84a362951001d7f6c31c': 'User_5', '8a0d5bed0152303242b9094ce32098e117fc8f1a7169e69df2cee99ad9d8ec3c': 'User_6', '586def0e79609c5554f12ed168ac1962c0d12e7d9ef87a91fdd02406c65dcd24': 'User_7', '0f26ab18818b468595373cc657917cfbbedacc14b7d4ff041a49b0aeb5a2f4c6': 'User_8', '00206cd4bdc39bab7e2908eb900370094d721883f30148868ba32abb8cdf28b3': 'User_9'})
raw_data.rename(columns={'Log ID': 'User Name'}, inplace=True)
raw_data.head()

,time,User Name,Machine ID,full_command,command,stderr,stdout
0,2021-04-19 19:34:00,User_1,10bcd15c,ls,ls,,SuperShell\nstderr.txt
1,2021-04-19 19:35:00,User_1,10bcd15c,export $TEST=Hello,export,/home/supershell/.offline.sh: line 358: export...,
2,2021-04-19 19:35:00,User_1,10bcd15c,export $TEST='Hello',export,/home/supershell/.offline.sh: line 358: export...,
3,2021-04-19 19:36:00,User_1,10bcd15c,export PATH=$PATH:$HOME/bin,export,,
4,2021-04-19 19:36:00,User_1,10bcd15c,export PATH=$PATH:test/bin,export,,


In [29]:
command_history = raw_data[["time", "User Name", "command", "full_command", "stderr"]]
command_history.head()

,time,User Name,command,full_command,stderr
0,2021-04-19 19:34:00,User_1,ls,ls,
1,2021-04-19 19:35:00,User_1,export,export $TEST=Hello,/home/supershell/.offline.sh: line 358: export...
2,2021-04-19 19:35:00,User_1,export,export $TEST='Hello',/home/supershell/.offline.sh: line 358: export...
3,2021-04-19 19:36:00,User_1,export,export PATH=$PATH:$HOME/bin,
4,2021-04-19 19:36:00,User_1,export,export PATH=$PATH:test/bin,


**Expand cell if there are multiple errors**

In [30]:
def error_splitter(errors):
    return errors.splitlines()

In [31]:
command_history["error list"] = raw_data["stderr"].apply(lambda x: error_splitter(x))
command_history.head(20)

<ipython-input-31-a433f53d6e6a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  command_history["error list"] = raw_data["stderr"].apply(lambda x: error_splitter(x))


,time,User Name,command,full_command,stderr,error list
0,2021-04-19 19:34:00,User_1,ls,ls,,[]
1,2021-04-19 19:35:00,User_1,export,export $TEST=Hello,/home/supershell/.offline.sh: line 358: export...,[/home/supershell/.offline.sh: line 358: expor...
2,2021-04-19 19:35:00,User_1,export,export $TEST='Hello',/home/supershell/.offline.sh: line 358: export...,[/home/supershell/.offline.sh: line 358: expor...
3,2021-04-19 19:36:00,User_1,export,export PATH=$PATH:$HOME/bin,,[]
4,2021-04-19 19:36:00,User_1,export,export PATH=$PATH:test/bin,,[]
5,2021-04-19 19:36:00,User_1,echo,echo $PATH,,[]
6,2022-09-29 18:52:00,User_2,clear,clear,,[]
7,2022-09-29 18:53:00,User_2,pwd,pwd,,[]
8,2022-09-29 18:53:00,User_2,echo,echo $PATH,,[]
9,2022-09-29 18:54:00,User_2,hostname,hostname,,[]


In [32]:
def error_expander(row):
    new = []
    error_list = row["error list"]
    if (len(error_list) <= 1):
        return new
    else:
        for error in error_list:
            time = row["time"]
            user_name = row["User Name"]
            command = row["command"]
            to_add = {
                "time": time,
                "User Name": user_name,
                "command": command,
                "stderr": error,
                "error list": error_list
            }
            new.append(to_add)
        return new

In [33]:
rows = command_history.apply(lambda x: error_expander(x), axis=1)

new = []
for row in rows:
    if (len(row) > 0):
        for i in range(len(row)):
            if (i > 0):
                new.append(row[i])

new[0:2]

[{'time': Timestamp('2022-09-11 19:17:00'),
  'User Name': 'User_3',
  'command': 'ls',
  'stderr': "ls: cannot access 'stdout.txt': No such file or directory",
  'error list': ["ls: cannot access '–l': No such file or directory",
   "ls: cannot access 'stdout.txt': No such file or directory"]},
 {'time': Timestamp('2022-09-13 13:32:00'),
  'User Name': 'User_3',
  'command': 'cat',
  'stderr': 'cat: cat: No such file or directory',
  'error list': ['cat: goodbye.txt: No such file or directory',
   'cat: cat: No such file or directory']}]

In [34]:
new_frame = pd.DataFrame.from_records(new)
error_expand = pd.concat([command_history, new_frame], ignore_index=True)
error_expand.sort_values(by=['time'], inplace=True)
error_expand.reset_index(drop=True, inplace=True)
error_expand.head(20)

,time,User Name,command,full_command,stderr,error list
0,2021-04-16 13:38:00,User_9,ls,ls,,[]
1,2021-04-16 13:39:00,User_9,cd,cd Bash,,[]
2,2021-04-16 13:39:00,User_9,ls,ls,,[]
3,2021-04-16 13:39:00,User_9,cd,cd ..,,[]
4,2021-04-16 13:39:00,User_9,cd,cd Bash,,[]
5,2021-04-16 13:39:00,User_9,mkdir,mkdir misc,,[]
6,2021-04-16 13:39:00,User_9,ls,ls,,[]
7,2021-04-16 13:40:00,User_9,echo,echo “hello world”,,[]
8,2021-04-16 13:41:00,User_9,echo,echo 1 2,,[]
9,2021-04-16 18:35:00,User_9,ls,ls,,[]


**Analyze on preceding commands of error**

In [35]:
file_name_replacer = re.compile(r'.+(\').+(\').+')
file_name = re.compile(r'(\').+(\')')

In [36]:
path_replacer = re.compile(r'.*(\/).+(: line )(\d+).*')
path_name = re.compile(r'(\/).+(: line )(\d+)')

In [37]:
name_between_colon = re.compile(r'.+(:).+(:).+')
colon_name = re.compile(r'(:).+(:)')

In [38]:
cd_replacer = re.compile(r'.*cd:(.+):.*')
cd_name = re.compile(r'cd:(.+):')

In [39]:
def replacer(error):
    if file_name_replacer.match(error):
        return re.sub(file_name, '$', error)
    elif path_replacer.match(error):
        error = re.sub(path_name, 'PATH', error)
        if file_name_replacer.match(error):
            return re.sub(file_name, '$', error)
        elif cd_replacer.match(error):
            return re.sub(cd_name, 'cd: $:', error)
        else:
            return error
    elif name_between_colon.match(error):
        return re.sub(colon_name, ': $:', error)

In [40]:
error_expand["no name"] = error_expand["stderr"].apply(lambda x: replacer(x))
error_expand.head()

,time,User Name,command,full_command,stderr,error list,no name
0,2021-04-16 13:38:00,User_9,ls,ls,,[],None
1,2021-04-16 13:39:00,User_9,cd,cd Bash,,[],None
2,2021-04-16 13:39:00,User_9,ls,ls,,[],None
3,2021-04-16 13:39:00,User_9,cd,cd ..,,[],None
4,2021-04-16 13:39:00,User_9,cd,cd Bash,,[],None


***Choose cat: cannot access $: No such file or directory as an example***

In [41]:
error_filter = (error_expand["no name"] == "cat: $: No such file or directory")

In [46]:
cat_error = error_expand[error_filter]
cat_error.head()

,time,User Name,command,full_command,stderr,error list,no name
68,2021-04-16 23:13:00,User_9,cat,"cat ""!!!!"" > goodbye.txt",cat: '!!!!': No such file or directory,[cat: '!!!!': No such file or directory],cat: $: No such file or directory
97,2021-04-17 13:50:00,User_9,cat,cat goodbye.txt,cat: goodbye.txt: No such file or directory,[cat: goodbye.txt: No such file or directory],cat: $: No such file or directory
172,2021-04-18 22:05:00,User_4,directory_grep_head_tail,NaN,cat: /home/jovyan/Bash/FilteredSortedOutput/Se...,[/home/jovyan/Bash/bin/directory_grep_head_tai...,cat: $: No such file or directory
176,2021-04-18 22:05:00,User_4,directory_grep_head_tail,NaN,cat: /home/jovyan/Bash/FilteredSortedOutput/Se...,[/home/jovyan/Bash/bin/directory_grep_head_tai...,cat: $: No such file or directory
410,2022-09-11 20:13:00,User_3,cat,cat hello.txt goodbye.txt,cat: goodbye.txt: No such file or directory,[cat: goodbye.txt: No such file or directory],cat: $: No such file or directory


In [48]:
cat_error_68 = error_expand.iloc[65:72]
cat_error_68

,time,User Name,command,full_command,stderr,error list,no name
65,2021-04-16 23:13:00,User_9,cat,cat goodbye.txt,,[],None
66,2021-04-16 23:13:00,User_9,cat,cat link_goodbye.txt,,[],None
67,2021-04-16 23:13:00,User_9,echo,"echo ""!!!!"" >> goodbye.txt",,[],None
68,2021-04-16 23:13:00,User_9,cat,"cat ""!!!!"" > goodbye.txt",cat: '!!!!': No such file or directory,[cat: '!!!!': No such file or directory],cat: $: No such file or directory
69,2021-04-16 23:15:00,User_9,cat,cat link_goodbye.txt,,[],None
70,2021-04-16 23:15:00,User_9,pico,NaN,,[],None
71,2021-04-16 23:15:00,User_9,diff,diff goodbye.txt copied_goodbye.txt,,[],None


In [49]:
cat_error_97 = error_expand.iloc[94:101]
cat_error_97

,time,User Name,command,full_command,stderr,error list,no name
94,2021-04-17 13:50:00,User_9,ls,ls -l,,[],None
95,2021-04-17 13:50:00,User_9,mv,mv goodbye.txt renamed_goodbye.txt,,[],None
96,2021-04-17 13:50:00,User_9,ls,ls -l,,[],None
97,2021-04-17 13:50:00,User_9,cat,cat goodbye.txt,cat: goodbye.txt: No such file or directory,[cat: goodbye.txt: No such file or directory],cat: $: No such file or directory
98,2021-04-17 13:57:00,User_9,ls,ls -r,,[],None
99,2021-04-17 13:57:00,User_9,ls,ls -l -r,,[],None
100,2021-04-17 13:57:00,User_9,ls,ls -R,,[],None


In [50]:
cat_error_410 = error_expand.iloc[407:414]
cat_error_410

,time,User Name,command,full_command,stderr,error list,no name
407,2022-09-11 20:12:00,User_3,pwd,pwd,,[],None
408,2022-09-11 20:13:00,User_3,cat,cat hello.txt goobye.txt,,[],None
409,2022-09-11 20:13:00,User_3,ls,ls,,[],None
410,2022-09-11 20:13:00,User_3,cat,cat hello.txt goodbye.txt,cat: goodbye.txt: No such file or directory,[cat: goodbye.txt: No such file or directory],cat: $: No such file or directory
411,2022-09-11 20:13:00,User_3,cat,cat hello.txt,,[],None
412,2022-09-11 20:20:00,User_3,cat,cat,,[],None
413,2022-09-11 20:22:00,User_3,ls,ls > ls.out,,[],None
